# Obtain a List of S3 URLs for a GES DISC Collection Using the CMR API
### Authors: Chris Battisto, Alexis Hunzinger
### Date Authored: 1-31-22
### Date Updated: 2-9-23

### Timing

Exercise: 15 minutes

### Overview

This notebook demonstrates how to obtain a list of S3 URLs for desired cloud-hosted GES DISC granules using the [Commmon Metadata Repository (CMR) API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).

### Prerequisites

This notebook was written using Python 3.8, and requires these libraries and files: 
- requests

Identify your data collection of interest and acquire its shortname or concept ID.
- These can be found on the collection's Dataset Landing Page on the GES DISC website. For example, this is the Dataset Landing Page for Hourly MERRA-2 SLV: https://disc.gsfc.nasa.gov/datasets/M2T1NXSLV_5.12.4/summary  
- Hover your mouse over the "Cloud Enabled" badge to find the collection's Concept ID, or find the collection's shortname in the Product Summary tab.

![](../../images/GESDISC-DSL-Shortname-ConceptID.png)


## Import Libraries

In [1]:
import requests

## Create a Function for CMR Catalog Requests

In [2]:
def cmr_request(params):
    response = requests.get(url,
                        params=params,
                        headers={
                            'Accept': 'application/json',
                        }
                       )
    return response

## Search CMR Catalogs and Obtain S3 URLs

### Check that the CMR catalog can be accessed

If "200, CMR is accessible" is returned, the catalog can be accessed!

In [3]:
url = 'https://cmr.earthdata.nasa.gov/search/collections'

# Create our request for finding cloud-hosted granules, and check that we can access CMR
response = cmr_request({
                        'cloud_hosted': 'True',
                        'has_granules': 'True'
                        })

if response.status_code == 200:
    print(str(response.status_code) + ", CMR is accessible")
else:
    print(str(response.status_code) + ", CMR is not accessible, check for outages")

200, CMR is accessible


### Search CMR with your desired data collection's shortname or concept ID and desired date range

Using the collection's shortname or concept ID, we can obtain individual granules by querying https://cmr.earthdata.nasa.gov/search/granules. By querying a JSON response of the granules that we want, we can obtain each granule's S3 URL. 

Here, we will parse out an S3 URL for the AQUA AIRS IR + MW Level 2 CLIMCAPS dataset. 
- **Shortname**: M2T1NXSLV
- **Concept ID**: C1276812863-GES_DISC

Our desried date range is September 25, 2022 00:00:00 to September 27, 2022 00:00:00. 

In [16]:
url = 'https://cmr.earthdata.nasa.gov/search/granules'

short_name = 'M2T1NXSLV'
concept_id = 'C1276812863-GES_DISC'

start_time = '2022-09-25T00:00:00Z'
end_time = '2022-09-27T00:00:00Z'

# OPTION 1: Using shortname
response = cmr_request({
                        'short_name': short_name,
                        'temporal': start_time+','+end_time,
                        'page_size': 200
                        })

# OPTION 2: Using concept ID
response = cmr_request({
                        'concept_id': concept_id,
                        'temporal': start_time+','+end_time,
                        'page_size': 200
                        })

granules = response.json()['feed']['entry']

### Identify the S3 URL from each granule response and save to a list

Now you can create an empty list and fill it with the S3 URL contained in each granule's response. 

In [17]:
s3_urls = []
for granule in granules:
    s3_urls.append(next((item['href'] for item in granule['links'] if item["href"].startswith("s3://")), None))

### View your list of S3 URLs

In [18]:
s3_urls

['s3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2022/09/MERRA2_400.tavg1_2d_slv_Nx.20220925.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2022/09/MERRA2_400.tavg1_2d_slv_Nx.20220926.nc4',
 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2022/09/MERRA2_400.tavg1_2d_slv_Nx.20220927.nc4']